<a href="https://colab.research.google.com/github/jee365/ESAA/blob/main/0624_%EB%B0%9C%ED%91%9C_%EB%AF%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **사물이미지 분류 경진대회 베이스라인2**

In [1]:
#필요한 library 들을 load
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_b3 as efficientnet
from sklearn.model_selection import train_test_split

## **데이터 로드 및 전처리**

In [2]:
#기본적인 전처리 코드
transform = transforms.Compose([
                                transforms.ToTensor(), #이미지 데이터를 tensor 데이터 포맷으로 변환
                                transforms.Resize([224,224]), #이미지의 크기를 통일
                                transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)) #픽셀 단위 데이터를 정규화
])

In [3]:
#데이터 로드 

from keras.datasets import cifar10
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [4]:
#train 데이터에서 validation 데이터를 나누어 줌
train_idx, valid_idx = train_test_split(np.arange(len(train_data)), test_size=0.2, random_state=42, shuffle=True, stratify=train_data.targets)

train_test_split에서 **stratify 값을 target으로 지정**해주면 각각의 class 비율을 **train / validation**로 유지 -> 성능 개선 (한 쪽에 쏠려서 분배되는 것을 방지)

In [5]:
batch_size = 32
num_workers = int(cpu_count()/2) #cpu_count로 cpu 개수 확인

In [6]:
#data loader 생성
#데이터 셋을 순회하며 모델에 데이터를 넣어주는 객체

train_loader = DataLoader(train_data, batch_size=batch_size, sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)

In [7]:
#데이터 크기 확인
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

In [8]:
print('total train imgs :',train_total,'/ total train batches :', train_batches)
print('total valid imgs :',valid_total, '/ total valid batches :', valid_batches)

total train imgs : 40000 / total train batches : 1250
total valid imgs : 10000 / total valid batches : 313


## **Device 설정**

Colab 런타임 > 런타임 유형변경에서 하드웨어 가속기를 **GPU**로 설정

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

## **모델 불러오기 / 파라미터 설정**

**efficientnet_b3** 모델 사용

**pretrained = False**로 설정해 사전 학습 모델 사용을 방지
- pretrained = True 면 ImageNet이라는 데이터셋을 대상으로 학습된 모델이 load
- pretrained = False 면 모델의 구조만 load 되고 모델의 가중치들은 load 되지 않음

In [10]:
net = models.efficientnet_b3(pretrained=False)
net.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

In [11]:
#모델의 마지막 layer의 output size와 분류할 라벨의 수를 입력
net.fc = nn.Linear(1000,10)
net = net.to(device)

In [12]:
#모델 파라미터 설정
criterion = CrossEntropyLoss() #loss 함수 지정
optimizer = optim.Adam(params=net.parameters(), lr=0.001) #optimizer 지정
epochs = 10

## **학습**

In [13]:
#반복문을 이용한 학습
for epoch in range(epochs):
  net.train() #모델을 호출하여 학습모드로 설정

  train_loss = 0
  train_correct = 0
  tqdm_dataset = tqdm(train_loader) #파이썬 작업 진행상황 표시
  for x,y in tqdm_dataset:
    x = x.to(device)
    y = y.to(device)
    outputs = net(x)
    loss = criterion(outputs, y)

    optimizer.zero_grad() #루프 한번 돌고나서 역전파를 하려면 zero_grad로 gradient 값들을 0으로 초기화
    loss.backward() #오차를 역전파 계산
    optimizer.step() #역전파 계산한 값으로 가중치를 수정

    train_loss += loss.item() #train_loss의 loss를 더함
    _, predicted = outputs.max(1) #언더바(_)를 이용해 해당 출력값은 저장하지 않음
    #각 열(axis = 1)마다 최댓값의 위치를 예측값으로 사용(확률이 가장 높은 레이블 계산)
    train_correct += predicted.eq(y).sum().item() #정답과 일치할 경우 train_correct를 증가

    tqdm_dataset.set_postfix({ #진행 상태 바 업데이트
        'Epoch': epoch+1,
        'Loss':'{:06f}'.format(loss.item()),
    })

  train_loss = train_loss / train_batches
  train_acc = train_correct / train_total

  net.eval() #신경망 추론 모드로 전환

  valid_loss = 0
  valid_correct = 0

  tqdm_dataset = tqdm(valid_loader)
  with torch.no_grad(): #autograd engine를 꺼서 자동으로 gradient를 트래킹 하지 않음
    for x,y in tqdm_dataset:
      x = x.to(device)
      y = y.to(device)

      outputs = net(x)
      loss = criterion(outputs, y)
      valid_loss += loss.item()
      _, predicted = outputs.max(1)
      valid_correct += predicted.eq(y).sum().item()

      tqdm_dataset.set_postfix({
        'Epoch': epoch+1,
        'Loss':'{:06f}'.format(loss.item()),
      })
      
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
    
    print('epochs',epoch+1, 'train loss',train_loss,'train acc', train_acc, 'valid loss',valid_loss, 'valid acc',valid_acc)


100%|██████████| 313/313 [00:29<00:00, 10.56it/s, Epoch=1, Loss=1.193166]


epochs 1 train loss 1.6569146045684815 train acc 0.39385 valid loss 1.2219117373323287 valid acc 0.5663


100%|██████████| 313/313 [00:29<00:00, 10.58it/s, Epoch=2, Loss=0.850322]


epochs 2 train loss 1.1189819053649903 train acc 0.6031 valid loss 0.946198442397407 valid acc 0.6658


100%|██████████| 313/313 [00:29<00:00, 10.52it/s, Epoch=3, Loss=1.053598]


...

epochs 8 train loss 0.3858802217274904 train acc 0.866075 valid loss 0.4411900221063687 valid acc 0.8615


100%|██████████| 313/313 [00:29<00:00, 10.52it/s, Epoch=9, Loss=0.504474]


epochs 9 train loss 0.33621820738315583 train acc 0.884825 valid loss 0.4093796618211384 valid acc 0.8635


100%|██████████| 313/313 [00:29<00:00, 10.51it/s, Epoch=10, Loss=0.434060]

epochs 10 train loss 0.29624942491352557 train acc 0.896925 valid loss 0.4126770795343783 valid acc 0.8642


## **모델 저장 / 불러오기**

In [14]:
#학습된 모델의 가중치를 저장
path = './model.pth'
torch.save(net.state_dict(),path) #state_dict는 각 계층을 매개변수 텐서로 매핑하는 Python 사전 객체

In [15]:
#저장된 모델을 불러오기
path = './model.pth'
net.load_state_dict(torch.load(path))

<All keys matched successfully>

## **추론**

In [16]:
from glob import glob
import PIL.Image
import numpy as np

from keras.datasets import cifar10
test_images = datasets.CIFAR10(root='./data/test', train=False, download=True, transform=transform)
test_images = np.array(test_images)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/test/cifar-10-python.tar.gz to ./data/test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [17]:
class CustomDataset(Dataset):
  def __init__(self, transform):
    self.transform = transform
    self.img_list = test_images
    self.img_labels = [0] * 10000

  def __len__(self):
    return len(self.img_list)

  def __getitem__(self, idx):
    return self.transform(self.img_list[idx]), self.img_labels[idx]

In [17]:
test_set = CustomDataset(transform)

In [17]:
test_loader = DataLoader(test_set, batch_size = batch_size, num_workers = num_workers)

현재 해당 대회의 데이터와 sample_submission 파일을 내려받을 수 없기에 sample_submission.csv를 채우는 코드는 생략

In [ ]:
import pandas as pd
sample_submission = pd.read_csv('./data/sample_submission.csv')

net.eval()

batch_index = 0

for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    outputs = net(images)
    batch_index = i * batch_size
    max_vals, max_indices = torch.max(outputs, 1)
    sample_submission.iloc[batch_index:batch_index + batch_size, 1:] = max_indices.long().cpu().numpy()[:,np.newaxis]

In [17]:
#예측된 데이터의 라벨을 숫자에서 딕셔너리 형태로 다시 복원
labels = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

sample_submission['target'] = sample_submission['target'].map(labels)

In [17]:
sample_submission.head()

In [17]:
sample_submission.to_csv('submit.csv',index=False)